# Classification Artificial Neural Networks (Attack)

### Importing libraries and reading data

In [5]:
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import tensorflow as tf
from sklearn.model_selection import KFold

In [6]:
df_train = pd.read_csv('df_train_attack.csv')
df_test = pd.read_csv('df_test_attack.csv')

In [7]:
X_train = df_train.drop('dist_BM_RA', axis=1)
y_train = df_train['dist_BM_RA']
X_test = df_test.drop(['dist_BM_RA'], axis=1)
y_test = df_test['dist_BM_RA']

In [10]:
round(1-y_train.value_counts()[1] / y_train.value_counts()[0],3)

0.851

### Scaling

In [35]:
scaler = StandardScaler()
#scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.values
y_test = y_test.values

### Training the Model

In [37]:
metrics = pd.DataFrame(columns=['fold', 'accuracy', 'precision', 'recall', 'f1_score'])

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, val in kfold.split(X_train, y_train):

    # Define the model architecture
    model = Sequential()

    model.add(Dense(units=36,activation='relu'))
    model.add(Dropout(0.2))

    #hidden layers
    model.add(Dense(units=18,activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=9,activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(units=1,activation='sigmoid'))

    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

    # Compile the model
    # For a binary classification problem
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(X_train[train], y_train[train],
          epochs=1000,
          batch_size=1,
          validation_data=(X_train[val], y_train[val]), 
          callbacks=[early_stop])

    # Generate generalization metrics
    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    scores = [fold_no,accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), 
              recall_score(y_test, y_pred), f1_score(y_test, y_pred)]

    metrics.loc[len(metrics)] = scores

    print(classification_report(y_test,y_pred))
    # Increase fold number
    fold_no = fold_no + 1


metrics.loc[len(metrics)] = ['mean', metrics['accuracy'].mean(), metrics['precision'].mean(), 
                             metrics['recall'].mean(), metrics['f1_score'].mean()]
metrics.loc[len(metrics)] = ['std', metrics['accuracy'].iloc[:-1].std(), metrics['precision'].iloc[:-1].std(), 
                             metrics['recall'].iloc[:-1].std(), metrics['f1_score'].iloc[:-1].std()]  
metrics = metrics.set_index('fold')

tz = pytz.timezone('Brazil/East')
metrics.to_csv( f'attack_{datetime.datetime.now(tz).strftime("%d-%m-%Y-%H:%M")}.csv')


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/1000
6800/6800 [==============================] - 12s 2ms/step - loss: 0.3555 - accuracy: 0.8672 - precision_13: 0.3682 - recall_13: 0.0447 - val_loss: 0.2053 - val_accuracy: 0.9012 - val_precision_13: 0.5811 - val_recall_13: 0.7963
Epoch 2/1000
6800/6800 [==============================] - 11s 2ms/step - loss: 0.2290 - accuracy: 0.9031 - precision_13: 0.6867 - recall_13: 0.4481 - val_loss: 0.1998 - val_accuracy: 0.9194 - val_precision_13: 0.7005 - val_recall_13: 0.6389
Epoch 3/1000
6800/6800 [==============================] - 11s 2ms/step - loss: 0.2004 - accuracy: 0.9138 - precision_13: 0.7379 - recall_13: 0.5596 - val_loss: 0.1829 - val_accuracy: 0.9200 - val_precision_13: 0.7326 - val_recall_13: 0.5833
Epoch 4/1000
6800/6800 [==============================] - 11s 2ms/step - loss: 0.1897 - accuracy: 0.9205 - precision_13: 0.7371 - recall_13: 0.6346 - val_loss: 0.1754 - val_accurac

In [38]:
metrics

,accuracy,precision,recall,f1_score
fold,,,,
1.0,0.921333,0.678571,0.707447,0.692708
2.0,0.929333,0.720430,0.712766,0.716578
3.0,0.901333,0.565359,0.920213,0.700405
4.0,0.933333,0.771605,0.664894,0.714286
5.0,0.924000,0.651639,0.845745,0.736111
mean,0.921867,0.677521,0.770213,0.712018
std,0.012386,0.077369,0.107862,0.016689
